In [4]:
import os, sys
import nltk
import urllib.request, urllib.error
import requests
from urllib.parse import urljoin
from bs4 import *
import datetime
import io
import re

In [2]:
def savePage(url, pagefilename='page'):
    def soupfindnSave(pagefolder, tag2find='img', inner='src'):
        """saves on specified `pagefolder` all tag2find objects"""
        if not os.path.exists(pagefolder): # create only once
            os.mkdir(pagefolder)
        for res in soup.findAll(tag2find):   # images, css, etc..
            try:         
                if not res.has_attr(inner): # check if inner tag (file object) exists
                    continue # may or may not exist
                filename = re.sub('\W+', '', os.path.basename(res[inner])) # clean special chars
                fileurl = urljoin(url, res.get(inner))
                filepath = os.path.join(pagefolder, filename)
                # rename html ref so can move html and folder of files anywhere
                res[inner] = os.path.join(os.path.basename(pagefolder), filename)
                if not os.path.isfile(filepath): # was not downloaded
                    with open(filepath, 'wb') as file:
                        filebin = session.get(fileurl)
                        file.write(filebin.content)
            except Exception as exc:
                print(exc, file=sys.stderr)
        return soup
    
    session = requests.Session()
    #... whatever other requests config you need here
    response = session.get(url)
    soup = BeautifulSoup(response.text, features="lxml")
    pagefolder = pagefilename
    soup = soupfindnSave(pagefolder, 'img', 'src')
    soup = soupfindnSave(pagefolder, 'link', 'href')
    soup = soupfindnSave(pagefolder, 'script', 'src')
    with open(pagefilename+'.html', 'wb') as file:
        file.write(soup.prettify('utf-8'))
    return soup

soup = savePage('https://en.wikipedia.org/wiki/Special:Random', 'wiki')

[Errno 2] No such file or directory: 'wiki\\'


In [3]:
url = 'https://en.wikipedia.org/wiki/Special:Random'

response = urllib.request.urlopen(url)
webContent = response.read()

# print(webContent)
soup = BeautifulSoup(webContent, 'html.parser')



css_files = []

for css in soup.find_all("link"):
    if css.attrs.get("href"):
        # if the link tag has the 'href' attribute
        css_url = urljoin(url, css.attrs.get("href"))
        css_files.append(css_url)


print("Total CSS files in the page:", len(css_files))

with open("css_files.txt", "w") as f:
    for css_file in css_files:
        print(css_file, file=f)
        
        
# print('\n'.join([str(i) for i in soup.find_all('p')]))
# print([i for i in soup.find_all('p')])

with io.open("output1.html", "w", encoding="utf-8") as file:
    file.write(str(soup.title) + '\n'.join([str(i) for i in soup.find_all('p')]))

Total CSS files in the page: 24


In [4]:
def crawl(pages, depth=None):
    indexed_url = [] # a list for the main and sub-HTML websites in the main website
    for i in range(depth):
        for page in pages:
            if page not in indexed_url:
                indexed_url.append(page)
                try:
                    c = urllib.request.urlopen(page)
                except:
                    print( "Could not open %s" % page)
                    continue
                soup = BeautifulSoup(c.read())
                links = soup('a') #finding all the sub_links
                for link in links:
                    if 'href' in dict(link.attrs):
                        url = urllib.parse.urljoin(page, link['href'])
                        if url.find("'") != -1:
                                continue
                        url = url.split('#')[0] 
                        if url[0:4] == 'http':
                                indexed_url.append(url)
        pages = indexed_url
    return indexed_url


In [5]:
pagelist=["https://en.wikipedia.org/wiki/Python_%28programming_language%29"]
urls = crawl(pagelist, depth=1)
# print( urls )

In [8]:
import requests

def copy_css(url, filename):
    response = requests.get(url)
    if response.status_code == 200:
        with open(filename, 'wb') as file:
            file.write(response.content)
        print(f"CSS file copied successfully as {filename}")
    else:
        print("Failed to copy CSS file")

# Example usage
url = 'http://example.com/style.css'
filename = 'style.css'
copy_css(url, filename)


Failed to copy CSS file


In [12]:
import requests
from bs4 import BeautifulSoup


def extract_css(url):
    response = requests.get(url)
    if response.status_code == 200:
        soup = BeautifulSoup(response.content, 'html.parser')
        css_files = []
        for css in soup.find_all("link", rel="stylesheet"):
            if css.attrs.get("href"):
                css_url = css.attrs.get("href")
                css_files.append(css_url)
        return css_files
    else:
        print("Failed to fetch webpage")

# Example usage
url = 'https://en.wikipedia.org/wiki/Fargo_(season_5)'
css_files = extract_css(url)
print(css_files)


['/w/load.php?lang=en&modules=codex-search-styles%7Cext.cite.styles%7Cext.uls.interlanguage%7Cext.visualEditor.desktopArticleTarget.noscript%7Cext.wikimediaBadges%7Cjquery.makeCollapsible.styles%7Cskins.vector.icons%2Cstyles%7Cskins.vector.zebra.styles%7Cwikibase.client.init&only=styles&skin=vector-2022', '/w/load.php?lang=en&modules=site.styles&only=styles&skin=vector-2022']


In [6]:
response = requests.get(url)


In [4]:
import requests
from bs4 import BeautifulSoup

def copy_style(url, output_file):
    response = requests.get(url)
    if response.status_code == 200:
        soup = BeautifulSoup(response.content, 'html.parser')
        css_files = []
        for css in soup.find_all("link", rel="stylesheet"):
            if css.attrs.get("href"):
                css_url = css.attrs.get("href")
                if css_url.startswith('http'):
                    css_files.append(css_url)
        if css_files:
            with open(output_file, 'w') as file:
                for css_url in css_files:
                    css_response = requests.get(css_url)
                    if css_response.status_code == 200:
                        file.write(css_response.text)
            print(f"Style copied successfully and saved to {output_file}")
        else:
            print("No CSS files found on the webpage")
    else:
        print("Failed to fetch webpage")

# Example usage
url = 'https://en.wikipedia.org/wiki/Fargo_(season_5)'
output_file = 'style.css'
copy_style(url, output_file)


No CSS files found on the webpage


In [7]:
response

<Response [404]>

In [5]:
import nltk
import random
from nltk import word_tokenize, pos_tag
from nltk.corpus import wordnet

def swap_nouns(text):
    # Tokenize the text into words
    words = word_tokenize(text)
    
    # Tag the words with their part-of-speech (POS)
    tagged_words = pos_tag(words)
    
    # Swap the nouns randomly
    for i, (word, tag) in enumerate(tagged_words):
        if tag.startswith('NN'):  # Check if the word is a noun
            synonyms = []
            for syn in wordnet.synsets(word):
                for lemma in syn.lemmas():
                    synonyms.append(lemma.name())
            if synonyms:
                random_synonym = random.choice(synonyms)
                tagged_words[i] = (random_synonym, tag)
    
    # Join the tagged words back into a string
    swapped_text = ' '.join(word for word, _ in tagged_words)
    
    return swapped_text

# Extract the contents of the webpage
url = 'https://en.wikipedia.org/wiki/Special:Random'
response = requests.get(url)
webContent = response.text

# Swap the nouns in the webpage content
swapped_content = swap_nouns(webContent)

# Print the swapped content
print(swapped_content)


LookupError: 
**********************************************************************
  Resource [93mpunkt[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('punkt')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mtokenizers/punkt/english.pickle[0m

  Searched in:
    - 'C:\\Users\\JIMBO-2020/nltk_data'
    - 'c:\\ProgramData\\Anaconda3\\nltk_data'
    - 'c:\\ProgramData\\Anaconda3\\share\\nltk_data'
    - 'c:\\ProgramData\\Anaconda3\\lib\\nltk_data'
    - 'C:\\Users\\JIMBO-2020\\AppData\\Roaming\\nltk_data'
    - 'C:\\nltk_data'
    - 'D:\\nltk_data'
    - 'E:\\nltk_data'
    - ''
**********************************************************************
